In [1]:
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from collections import OrderedDict

In [2]:
#getting the client id and secret
clientid=''#Enter your cliend ID here
clientsecret=''#Enter your cliend secret here
authurl='https://accounts.spotify.com/api/token'
response=requests.post(authurl,{
    'grant_type':'client_credentials',
    'client_id':clientid,
    'client_secret':clientsecret,
})
authjson=response.json()
accesstoken=authjson['access_token']
headers={'Authorization':'Bearer {}'.format(accesstoken)}

In [3]:
#using the playlists' songs, we create dataframes with details of the songs, and an extra column, genre
def playlistdataframe(playlistid,genre):    
    base='https://api.spotify.com/v1/playlists/'
    r=requests.get(base+playlistid+'/tracks',headers=headers)
    playlist=r.json()
    ids=[]
    for song in playlist['items']:
        ids.append(song['track']['id'])
    playlistids=",".join(ids)
    r=requests.get("https://api.spotify.com/v1/audio-features/?ids={}".format(playlistids), headers=headers)
    details=r.json()
    df=pd.DataFrame(details['audio_features'])
    df['genre']=genre
    return df

In [39]:
rock1=playlistdataframe('37i9dQZF1DXcF6B6QPhFDv','rock')
rock2=playlistdataframe('37i9dQZF1DWWJOmJ7nRx0C','rock')
#rock3=playlistdataframe('37i9dQZF1DX82GYcclJ3Ug','rock')
rock4=playlistdataframe('37i9dQZF1DWXRqgorJj26U','rock')
hiphop1=playlistdataframe('37i9dQZF1DX0XUsuxWHRQd','hip hop')
hiphop2=playlistdataframe('37i9dQZF1DWVA1Gq4XHa6U','hip hop')
hiphop3=playlistdataframe('37i9dQZF1DX186v583rmzp','hip hop')
# hiphop4=playlistdataframe('37i9dQZF1DWT5MrZnPU1zD','hip hop')
#edm1=playlistdataframe('37i9dQZF1DX4dyzvuaRJ0n','electronic')
edm2=playlistdataframe('37i9dQZF1DXaXB8fQg7xif','electronic')
edm3=playlistdataframe('37i9dQZF1DWXLeA8Omikj7','electronic')
edm4=playlistdataframe('37i9dQZF1DX6VdMW310YC7','electronic')
class1=playlistdataframe('37i9dQZF1DWWEJlAGA9gs0','classical')
class2=playlistdataframe('37i9dQZF1DWV0gynK7G6pD','classical')
class3=playlistdataframe('37i9dQZF1DWVFeEut75IAL','classical')
pop1=playlistdataframe('37i9dQZF1DWUa8ZRTfalHk','pop')
pop2=playlistdataframe('37i9dQZF1DX5gQonLbZD9s','pop')
pop3=playlistdataframe('37i9dQZF1DX0s5kDXi1oC5','pop')
metal1=playlistdataframe('37i9dQZF1DWWOaP4H0w5b0','metal')
metal2=playlistdataframe('37i9dQZF1DXakaomPRkkDa','metal')
metal3=playlistdataframe('37i9dQZF1DX9qNs32fujYe','metal')

In [40]:
#concatenation to make a mega dataframe
musicdf=pd.concat([rock1,rock2,rock4,hiphop1,hiphop2,hiphop3,edm2,edm3,edm4,class1,class2,class3,pop1,pop2,pop3,metal1,metal2,metal3])
musicdf=musicdf.drop_duplicates(subset=['id'],keep='first')
musicdf.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,0.530,0.759,7,-7.067,1,0.0351,0.009840,0.000000,0.319,0.502,131.999,audio_features,3NUmUIyzNLBp8bCFMH8Mif,spotify:track:3NUmUIyzNLBp8bCFMH8Mif,https://api.spotify.com/v1/tracks/3NUmUIyzNLBp...,https://api.spotify.com/v1/audio-analysis/3NUm...,253840,4,rock
1,0.394,0.833,0,-5.180,1,0.0420,0.008630,0.811000,0.109,0.362,145.082,audio_features,55meRTYBw8S5q7KF3DkjL7,spotify:track:55meRTYBw8S5q7KF3DkjL7,https://api.spotify.com/v1/tracks/55meRTYBw8S5...,https://api.spotify.com/v1/audio-analysis/55me...,250961,4,rock
2,0.471,0.900,8,-2.283,0,0.0832,0.014800,0.000003,0.082,0.362,91.318,audio_features,03Szk0skbXqllHkNCVZI9p,spotify:track:03Szk0skbXqllHkNCVZI9p,https://api.spotify.com/v1/tracks/03Szk0skbXql...,https://api.spotify.com/v1/audio-analysis/03Sz...,270215,4,rock
3,0.475,0.844,1,-4.448,0,0.0622,0.000120,0.111000,0.221,0.554,156.136,audio_features,4CEqDmkqD7illXHuym76V6,spotify:track:4CEqDmkqD7illXHuym76V6,https://api.spotify.com/v1/tracks/4CEqDmkqD7il...,https://api.spotify.com/v1/audio-analysis/4CEq...,246933,4,rock
4,0.185,0.900,4,-5.414,0,0.0918,0.000125,0.020500,0.119,0.240,159.813,audio_features,3WEerYUbUTVFCWE3gcADiz,spotify:track:3WEerYUbUTVFCWE3gcADiz,https://api.spotify.com/v1/tracks/3WEerYUbUTVF...,https://api.spotify.com/v1/audio-analysis/3WEe...,241857,4,rock


In [41]:
#separating the independent variables from the dependent
X=musicdf[['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence']]
y=musicdf.genre
%store X
%store y

Stored 'X' (DataFrame)
Stored 'y' (Series)


In [42]:
#standardizing the values to make better predictions
standardized_df=StandardScaler().fit_transform(X)
scaledmusicdf=pd.DataFrame(X,columns=X.columns)
scaledmusicdf.head()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence
0,0.009840,0.530,0.759,0.000000,7,0.319,-7.067,0.0351,131.999,4,0.502
1,0.008630,0.394,0.833,0.811000,0,0.109,-5.180,0.0420,145.082,4,0.362
2,0.014800,0.471,0.900,0.000003,8,0.082,-2.283,0.0832,91.318,4,0.362
3,0.000120,0.475,0.844,0.111000,1,0.221,-4.448,0.0622,156.136,4,0.554
4,0.000125,0.185,0.900,0.020500,4,0.119,-5.414,0.0918,159.813,4,0.240


In [43]:
#split up of the training and testing data
X_train, X_test, y_train, y_test=train_test_split(scaledmusicdf, y, test_size=0.2, random_state=4)

In [10]:
#KNN: finding the best k value
acc=0
best_k=0
ypred=0
for i in range(1, 50):
    knn=KNeighborsClassifier(n_neighbors=i).fit(X_train, y_train)
    yhat=knn.predict(X_test)
    knnacc=accuracy_score(y_test, yhat)
    if knnacc>acc:
        acc=knnacc
        best_k=i
        ypred=yhat
print("Best accuracy: {}\nBest K value: {}".format(acc, best_k))
knnpred=KNeighborsClassifier(n_neighbors=best_k)
knnpred.fit(X_train, y_train)
print(knnpred.score(X_test, y_test))
print(precision_recall_fscore_support(y_test, ypred, average='micro'))
print(confusion_matrix(y_test, ypred))
print(classification_report(y_test,ypred))

Best accuracy: 0.47750865051903113
Best K value: 11
0.47750865051903113
(0.47750865051903113, 0.47750865051903113, 0.47750865051903113, None)
[[42  6  0  0  0  0]
 [ 7 36  2  3  6  3]
 [ 0  2 18  7  6  3]
 [ 0  8  8 20  8  4]
 [ 0  8 11 11 12  3]
 [ 2  8 10 19  6 10]]
              precision    recall  f1-score   support

   classical       0.82      0.88      0.85        48
  electronic       0.53      0.63      0.58        57
     hip hop       0.37      0.50      0.42        36
       metal       0.33      0.42      0.37        48
         pop       0.32      0.27      0.29        45
        rock       0.43      0.18      0.26        55

    accuracy                           0.48       289
   macro avg       0.47      0.48      0.46       289
weighted avg       0.47      0.48      0.46       289



In [44]:
#random forest classification
forestpred=RandomForestClassifier(n_estimators=100, max_depth=6)
forestpred.fit(X_train, y_train)
print(forestpred.score(X_test, y_test))
randomforest=forestpred.predict(X_test)
print(confusion_matrix(y_test,randomforest))
print(classification_report(y_test, randomforest))

0.7370242214532872
[[46  1  0  0  0  0]
 [ 2 46  0  0  8  1]
 [ 0  0 31  1  4  0]
 [ 0  0  0 39  0  9]
 [ 0  2  8  0 33  3]
 [ 0  5  3 18 11 18]]
              precision    recall  f1-score   support

   classical       0.96      0.98      0.97        47
  electronic       0.85      0.81      0.83        57
     hip hop       0.74      0.86      0.79        36
       metal       0.67      0.81      0.74        48
         pop       0.59      0.72      0.65        46
        rock       0.58      0.33      0.42        55

    accuracy                           0.74       289
   macro avg       0.73      0.75      0.73       289
weighted avg       0.73      0.74      0.72       289



In [12]:
#logistic regression classification
logpred=LogisticRegression(solver='lbfgs', max_iter=10000)
logpred.fit(X_train, y_train)
logpredict=logpred.predict(X_test)
print(logpred.score(X_test, y_test))
print(confusion_matrix(y_test,logpredict))
print(classification_report(y_test, logpredict))

0.698961937716263
[[47  0  0  0  1  0]
 [ 5 36  4  4  8  0]
 [ 0  0 28  0  8  0]
 [ 0  1  0 41  0  6]
 [ 0  0  5  5 30  5]
 [ 0  5  2 19  9 20]]
              precision    recall  f1-score   support

   classical       0.90      0.98      0.94        48
  electronic       0.86      0.63      0.73        57
     hip hop       0.72      0.78      0.75        36
       metal       0.59      0.85      0.70        48
         pop       0.54      0.67      0.59        45
        rock       0.65      0.36      0.47        55

    accuracy                           0.70       289
   macro avg       0.71      0.71      0.70       289
weighted avg       0.71      0.70      0.69       289



In [13]:
#SVM
svmpred=SVC(kernel='linear')
svmpred.fit(X_train, y_train)
svmpred.score(X_test, y_test)

0.7093425605536332

In [14]:
#neural networks
nnpred=MLPClassifier(hidden_layer_sizes=600)
nnpred.fit(X_train, y_train)
nnpred.score(X_test, y_test)

0.671280276816609

In [15]:
#summary of the used classifiers with 10 instances
classifiers=[knnpred, forestpred, logpred, svmpred, nnpred]
modelresult=[]
for i in classifiers:
    modelresult.append(cross_val_score(i, X_train, y_train, scoring='accuracy', cv=10))
modeldf=pd.DataFrame(modelresult, columns=[x for x in range(1,11)], index=["KNN", "Random Forest", "Logistic", "SVM", "NN"])
modeldf["Mean"] = modeldf.mean(axis=1)
modeldf

,1,2,3,4,5,6,7,8,9,10,Mean
KNN,0.413793,0.474138,0.508621,0.491379,0.486957,0.478261,0.539130,0.521739,0.513043,0.504348,0.493141
Random Forest,0.681034,0.758621,0.818966,0.750000,0.782609,0.730435,0.800000,0.765217,0.739130,0.730435,0.755645
Logistic,0.663793,0.698276,0.741379,0.715517,0.773913,0.695652,0.765217,0.730435,0.695652,0.721739,0.720157
SVM,0.655172,0.732759,0.732759,0.775862,0.773913,0.669565,0.730435,0.747826,0.721739,0.721739,0.726177
NN,0.594828,0.620690,0.577586,0.655172,0.713043,0.600000,0.669565,0.530435,0.678261,0.608696,0.624828


In [16]:
#hyperparameter tuning for knn
knn=KNeighborsClassifier(n_neighbors=best_k, n_jobs=-1)
params={'leaf_size':[1,2,3,4,5], 'weights':['uniform','distance'], 'algorithm':['auto','ball_tree','brute'],'n_jobs':[-1]}
knnmodel=GridSearchCV(knn, param_grid=params,n_jobs=1)
knnmodel.fit(X_train, y_train)
knnbest=knnmodel.best_params_
print(knnbest)
print(f"Training best: {knnmodel.score(X_train, y_train)}\nTesting best: {knnmodel.score(X_test, y_test)}")

{'algorithm': 'auto', 'leaf_size': 1, 'n_jobs': -1, 'weights': 'uniform'}
Training best: 0.5831889081455806
Testing best: 0.47750865051903113


In [45]:
#hyperparameter tuning for random forest
forest=RandomForestClassifier()
params={'criterion':['gini', 'entropy'],'n_estimators':[5,10,15,20,25], 'min_samples_leaf':[1,2,3],
        'min_samples_split':[3,4,5,6,7],'random_state':[123],'n_jobs':[-1]}
forestmodel=GridSearchCV(forest,param_grid=params,n_jobs=-1)
forestmodel.fit(X_train,y_train)
forestbest=forestmodel.best_params_
print(forestbest)
print(f"Training best: {forestmodel.score(X_train, y_train)}\nTesting best: {forestmodel.score(X_test, y_test)}")

{'criterion': 'entropy', 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 25, 'n_jobs': -1, 'random_state': 123}
Training best: 0.9774696707105719
Testing best: 0.7335640138408305


In [20]:
#hyperparameter tuning for logistic regression
logregression=LogisticRegression(max_iter=100000)
params={'C':np.logspace(-3,3,7), 'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'penalty':['l2']}
logmodel=GridSearchCV(logregression, param_grid=params, cv=3)
logmodel.fit(X_train, y_train)
logbest=logmodel.best_params_
print(logbest)
print(f"Training best: {logmodel.score(X_train, y_train)}\nTesting best: {logmodel.score(X_test, y_test)}")

{'C': 100.0, 'penalty': 'l2', 'solver': 'lbfgs'}
Training best: 0.7608318890814558
Testing best: 0.7197231833910035


In [21]:
#hyperparameter tuning for SVM
svm=SVC()
params={'C':[0.001, 0.01, 0.1, 1, 10], 'gamma':[0.001, 0.01, 0.1, 1], 'kernel': ['linear', 'rbf']} #poly and sigmoid take too long
svmmodel=GridSearchCV(svm, param_grid=params, refit=True, scoring='accuracy', verbose=10, cv=3, n_jobs=-1)
svmmodel.fit(X_train, y_train)
svmbest=svmmodel.best_params_
print(svmbest)
print(f"Training best: {svmmodel.score(X_train, y_train)}\nTesting best: {svmmodel.score(X_test, y_test)}")

Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1936s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done  62 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Batch computation too slow (2.2212s.) Setting batch_size=1.
[Parallel(n_jobs=-1)]: Done  84 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 103 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   50.4s finished


{'C': 10, 'gamma': 0.001, 'kernel': 'linear'}
Training best: 0.7755632582322357
Testing best: 0.71280276816609


In [22]:
#hyperparameter tuning for neural networks
nn=MLPClassifier()
params = {'alpha': 10.0**(-np.arange(1, 10)), 'hidden_layer_sizes':[200, 400, 600, 800, 1000], 
          'activation': ['relu', 'tanh', 'logistic']}
nnmodel=GridSearchCV(nn, param_grid=params, n_jobs=-1, scoring='accuracy', cv=3)
nnmodel.fit(X_train, y_train)
nnbest=nnmodel.best_params_
print(nnbest)
print(f"Training best: {nnmodel.score(X_train, y_train)}\nTesting best: {nnmodel.score(X_test, y_test)}")

{'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': 400}
Training best: 0.7530329289428076
Testing best: 0.7093425605536332


In [26]:
#summary of the best versions of each classifier for 10 instances
knnfinal=KNeighborsClassifier(leaf_size=knnbest['leaf_size'], n_neighbors=21, algorithm=knnbest['algorithm'], 
                              weights=knnbest['weights'])
forestfinal=RandomForestClassifier(criterion=forestbest['criterion'], min_samples_split=forestbest['min_samples_split'], 
                                   min_samples_leaf=forestbest['min_samples_leaf'], n_estimators=forestbest['n_estimators'],
                                   random_state=0)
logfinal=LogisticRegression(max_iter=100000, C=logbest['C'], solver=logbest['solver'])
svmfinal=SVC(kernel=svmbest['kernel'], gamma=svmbest['gamma'], C=svmbest['C'])
nnfinal=MLPClassifier(activation=nnbest['activation'], alpha=nnbest['alpha'], hidden_layer_sizes=nnbest['hidden_layer_sizes'])


classifiers=[knnfinal, forestfinal, logfinal, svmfinal, nnfinal]
results=[]
for i in classifiers:
    results.append(cross_val_score(i, X_train, y_train, scoring='accuracy', cv=3))
finaldf=pd.DataFrame(results, columns=[x for x in range(1,4)], index=["KNN", "Random Forest", "Logistic", "SVM", "NN"])
finaldf["Mean"] = finaldf.mean(axis=1)
finaldf
#Random Forest is the best, so let's use it for predictions
#future scope would involve boosting

c:\users\lenovo\appdata\local\programs\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\users\lenovo\appdata\local\programs\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\users\lenovo\appdata\local\programs\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,1,2,3,Mean
KNN,0.501299,0.514286,0.460938,0.492174
Random Forest,0.735065,0.766234,0.739583,0.746961
Logistic,0.740260,0.732468,0.752604,0.741777
SVM,0.735065,0.761039,0.742188,0.746097
NN,0.703896,0.696104,0.731771,0.710590


In [27]:
#insert playlist id
trial=playlistdataframe('3aFp6OcR1rukcTX0A8ppxe',0)

In [51]:
#extracting categories from the mega dataframe
categories=pd.factorize(musicdf['genre'])[1]
converter=OrderedDict()
for i in range(len(categories)):
    converter[i]=categories[i]

Index(['rock', 'hip hop', 'electronic', 'classical', 'pop', 'metal'], dtype='object') OrderedDict([(0, 'rock'), (1, 'hip hop'), (2, 'electronic'), (3, 'classical'), (4, 'pop'), (5, 'metal')])


In [46]:
#factorizing the genre column; making it numerical for categorization
genres=pd.factorize(y)[0]
forestfinal.fit(X, genres)

RandomForestClassifier(min_samples_split=7, n_estimators=25, random_state=0)

In [47]:
#predicting genres for the given features of the playlist's songs
finalX=trial[['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence']]
predictions=forestfinal.predict(finalX)

In [48]:
#probabilities of the predictions
predprob=forestfinal.predict_proba(finalX)
print(predprob)

[[0.32068254 0.         0.039      0.         0.         0.64031746]
 [0.02       0.42247619 0.16142857 0.         0.38442857 0.01166667]
 [0.3611746  0.         0.04       0.         0.         0.5988254 ]
 [0.29139394 0.00571429 0.23733766 0.00571429 0.3480303  0.11180952]
 [0.25042857 0.09842857 0.04785714 0.         0.54757143 0.05571429]
 [0.14471429 0.         0.         0.         0.04571429 0.80957143]
 [0.01333333 0.04       0.04357143 0.         0.89642857 0.00666667]
 [0.44929437 0.02666667 0.00666667 0.         0.27466667 0.24270563]
 [0.12209524 0.03180952 0.03333333 0.         0.09047619 0.72228571]
 [0.25689177 0.         0.01071429 0.         0.         0.73239394]]


In [49]:
#converting the numerical predictions into categorical
predicted_genres = [converter[prediction] for prediction in predictions]
trial['genre']=predicted_genres

In [50]:
print(trial)

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.513   0.880    7    -4.690     0       0.0309      0.000096   
1         0.712   0.556    5    -7.214     0       0.0531      0.084000   
2         0.232   0.937    4    -4.141     1       0.0998      0.003890   
3         0.494   0.723    6    -4.559     0       0.0441      0.019200   
4         0.465   0.735    6    -3.715     1       0.1170      0.071100   
5         0.402   0.976    9    -1.823     1       0.0712      0.002900   
6         0.640   0.533    0    -6.596     1       0.0706      0.119000   
7         0.535   0.910   10    -4.260     1       0.0511      0.014000   
8         0.526   0.979    3    -5.069     0       0.1460      0.002640   
9         0.511   0.972   10    -3.751     1       0.0413      0.000005   

   instrumentalness  liveness  valence    tempo            type  \
0          0.006140    0.0824    0.495   92.358  audio_features   
1          0.000050    0.5270    0.220  